In [70]:
#notes
#{id: '', source: '', created_at: '', text: '', metadata: {'path':, type :'note'},  }

#IMPORTS
import json
import hashlib
import re
from pathlib import Path
from datetime import datetime, timezone
import requests

import sys
from dotenv import load_dotenv
import os
load_dotenv()
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
if not NOTION_TOKEN:
    print("Missing NOTION_TOKEN in .env")
    sys.exit(1)

NOTION_DB_ID = os.environ.get("NOTION_DB_ID")
if not NOTION_DB_ID:
    print("Missing NOTION_DB_ID in .env")
    sys.exit(1)


#NORMALIZE
def normalize_text(text: str) -> str:

    # This will convert win and old mac format to \n
    # \r\n is format from win
    # \r is format from mac
    text = text.replace('\r\n', '\n').replace('\r','\n')

    lines = text.split('\n')

    # delete blank line in top and bottom of content
    while lines and lines[0].strip() == "":
        lines.pop(0)
    while lines and lines[-1].strip() == "":
        lines.pop()

    # delete blank line between content
    normalized = []
    blank = False
    for line in lines:
        if line.strip() == "":
            if not blank:
                normalized.append("")
            blank=True
        else:
            normalized.append(line.rstrip())
            blank=False

    return '\n'.join(normalized)



header = {
    'Authorization': f'Bearer {NOTION_TOKEN}',
    'Notion-Version': '2022-06-28',
    'Content-Type': 'application/json'
}

link = f"https://api.notion.com/v1/databases/{NOTION_DB_ID}/query"
r = requests.post(link, headers=header, json={'page_size':5})
# print(r.text)
print(r.status_code)




200


In [71]:
# print(type(r.json()))
for it in r.json().items():
    # if type(it[1]) == dict():
    #     for i in it.values():
    #         print(i)
    print('---')
    print(it)

print(r.json().keys())
print(r.json().get('results', []))


---
('object', 'list')
---
('results', [{'object': 'page', 'id': '2e315ceb-cf59-805e-a0bc-e5da60cc8acd', 'created_time': '2026-01-09T07:13:00.000Z', 'last_edited_time': '2026-01-10T04:28:00.000Z', 'created_by': {'object': 'user', 'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'}, 'last_edited_by': {'object': 'user', 'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '2e315ceb-cf59-8057-8ea4-f8576634b23b'}, 'archived': False, 'in_trash': False, 'is_locked': False, 'properties': {'Created time': {'id': 'fZlU', 'type': 'created_time', 'created_time': '2026-01-09T07:13:00.000Z'}, 'Date': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': '1/9/2026', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': '1/9/2026', 'href': None}]}}, 'url': 'https://www.notion.so/1-9-2026-2e315cebcf59805ea0bce

In [73]:
# r.json().get('results', [])[0].get('properties', {}).get('Date', {})[title]

In [74]:
created_time = []
titles = []
for page in r.json().get('results', []):
    created_time.append(page['created_time'])
    title = "".join(item.get("plain_text","") for item in page['properties']['Date']['title'])
    titles.append(title)
print(created_time, titles)

['2026-01-09T07:13:00.000Z', '2026-01-10T08:03:00.000Z', '2026-01-10T04:28:00.000Z', '2026-01-10T08:03:00.000Z'] ['1/9/2026', ' these links', '1/10/2026', '']


In [82]:
r.json().get('results', [])[0]
block_id = r.json().get('results', [])[0]['id']


In [83]:
print(block_id)

2e315ceb-cf59-805e-a0bc-e5da60cc8acd


In [87]:
page_link = f'https://api.notion.com/v1/blocks/{block_id}/children'
page_r = requests.get(page_link, headers=header)
page_r.status_code

200

In [101]:
page_r.json()['results'][0]
# for block in page_r.json()['results']:
#     print(block['type'])

{'object': 'block',
 'id': '2e315ceb-cf59-808f-825c-ed950d4a0bc6',
 'parent': {'type': 'page_id',
  'page_id': '2e315ceb-cf59-805e-a0bc-e5da60cc8acd'},
 'created_time': '2026-01-09T07:14:00.000Z',
 'last_edited_time': '2026-01-09T10:41:00.000Z',
 'created_by': {'object': 'user',
  'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'},
 'last_edited_by': {'object': 'user',
  'id': '7e6073e6-8de6-47ab-9855-e9b28e8ed05f'},
 'has_children': False,
 'archived': False,
 'in_trash': False,
 'type': 'quote',
 'quote': {'rich_text': [{'type': 'text',
    'text': {'content': 'Endtime/ Hours spent: 17h30/3h30', 'link': None},
    'annotations': {'bold': False,
     'italic': False,
     'strikethrough': False,
     'underline': False,
     'code': False,
     'color': 'default'},
    'plain_text': 'Endtime/ Hours spent: 17h30/3h30',
    'href': None}],
  'color': 'default'}}

In [ ]:
def block_to_text(block):

    if block['type'] == 'quote':
        return  block['quote']['plain_text']
        

In [ ]:
page_link = f'https://api.notion.com/v1/blocks/{block_id}/children'
page_r = requests.get(page_link, headers=header)
page_r.status_code

200